# Analyze metadata semantically with an LLM 🤖

This script retrieves a list of open government datasets from an online metadata API, filters them to keep only certain entries, and then uses an OpenAI language model to perform a semantic analysis of each dataset’s title and description. The analysis generates several scores (content, context, quality, and spatial), which are then combined with the original metadata. Finally, the script saves this enriched information to an Excel file, delivering an AI-driven summary of each dataset’s metadata. 🚀


In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import pandas as pd
import numpy as np

# Display a message to inform the user about the import process
print("Importing necessary libraries and setting options...")

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500

from datetime import datetime
from tqdm.notebook import tqdm
import requests
from concurrent.futures import ThreadPoolExecutor
from utils import do_full_analysis, parse_analysis_results, parse_instructor_results
import warnings
import time
import json
import re

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500

warnings.simplefilter("ignore", category=(UserWarning, FutureWarning))

Importing necessary libraries and setting options...


In [3]:
# Sanity check whether the API key is set

import os
import openai

# Retrieve the API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY is not set in the environment.")
else:
    print("API key is correctly set.")

# Initialize the OpenAI client with the API key
openai.api_key = OPENAI_API_KEY

# Example usage
OPENAI_SYSTEM_MESSAGE = "You are a helpful assistant."

API key is correctly set.


In [3]:
# Constants
# Dataset links are composed of this baselink and the identifier for each dataset.
BASELINK_DATASHOP = "https://daten.berlin.de/datensaetze/"

MDV_DATA_PATH = "_data/01_mdv_metadata.parq"

# Default figure size for Quarto HTML output.
FIGSIZE = (7, 5)

# Print output to inform the user about the constants being set
print("Constants have been defined:")
print(f"BASELINK_DATASHOP: {BASELINK_DATASHOP}")
print(f"MDV_DATA_PATH: {MDV_DATA_PATH}")
print(f"FIGSIZE: {FIGSIZE}")

Constants have been defined:
BASELINK_DATASHOP: https://daten.berlin.de/datensaetze/
MDV_DATA_PATH: _data/01_mdv_metadata.parq
FIGSIZE: (7, 5)


In this notebook we perform a specific semantic analysis of titles and descriptions in our metadata catalog with LLMs.


## Load data


## Retrieve data from the metadata API.

The code snippet you provided retrieves metadata from an API and expects the JSON response to contain a key named `"dataset"`. This key should map to a list of dataset entries, which can be normalized into a DataFrame using `pd.json_normalize`.

### Expected JSON Format

The expected format of the JSON response from the API should look something like this:

```json
{
  "dataset": [
    {
      "identifier": "dataset1",
      "title": "Dataset Title 1",
      "description": "Description of dataset 1",
      "publisher": {
        "name": "Publisher Name"
      },
      "keyword": ["keyword1", "keyword2"]
      // other fields...
    },
    {
      "identifier": "dataset2",
      "title": "Dataset Title 2",
      "description": "Description of dataset 2",
      "publisher": {
        "name": "Publisher Name"
      },
      "keyword": ["keyword3", "keyword4"]
      // other fields...
    }
    // more datasets...
  ]
}
```

### Key Points

- The top-level key should be `"dataset"`, which contains an array of dataset objects.
- Each dataset object should have fields like `"identifier"`, `"title"`, `"description"`, and `"publisher"`, among others.
- The `pd.json_normalize` function will flatten nested structures, so if there are nested objects (like `"publisher"`), it will create separate columns for their fields.


In [4]:
DATA_PATH = "metadata.parquet"

In [ ]:
# Constants
MDV_API_LINK = (
    "https://datenregister.berlin.de/api/3/action/current_package_list_with_resources"
)


def get_full_package_list(limit=500, sleep=2):
    """Get full package list from CKAN API"""
    offset = 0
    frames = []
    while True:
        print(f"{offset} packages retrieved.")
        url = MDV_API_LINK + f"?limit={limit}&offset={offset}"
        res = requests.get(url)
        data = res.json()
        if data["result"] == []:
            break
        data = pd.DataFrame(pd.json_normalize(data["result"]))
        frames.append(data)
        offset += limit
        time.sleep(sleep)

    data = pd.concat(frames)
    data = data.reset_index(drop=True)

    # Convert complex objects to strings for safe parquet storage
    object_cols = data.select_dtypes(include=["object"]).columns
    for col in object_cols:
        if data[col].notna().any() and isinstance(data[col].iloc[0], (list, dict)):
            data[col] = data[col].apply(json.dumps)

    return data


# Retrieve metadata for all datasets
df = get_full_package_list()

# Save to parquet
df.to_parquet(DATA_PATH)

# Print the path and file that was saved
print(f"Saved the dataset to: {DATA_PATH}")

# Give user some info about the datasets
print(
    f"We have {len(df):,.0f} datasets in the catalogue and {df.shape[1]} properties.\n"
)
display(df.info(memory_usage="deep"))
display(df.head())

In [6]:
df.columns

Index(['author', 'author_email', 'berlin_source', 'berlin_type',
       'creator_user_id', 'date_released', 'date_updated',
       'geographical_coverage', 'geographical_granularity', 'id', 'isopen',
       'license_id', 'license_title', 'license_url', 'maintainer',
       'maintainer_email', 'metadata_created', 'metadata_modified', 'name',
       'notes', 'num_resources', 'num_tags', 'owner_org', 'private', 'state',
       'temporal_coverage_from', 'temporal_coverage_to',
       'temporal_granularity', 'title', 'type', 'url', 'version', 'groups',
       'resources', 'tags', 'relationships_as_subject',
       'relationships_as_object', 'organization.id', 'organization.name',
       'organization.title', 'organization.type', 'organization.description',
       'organization.image_url', 'organization.created',
       'organization.is_organization', 'organization.approval_status',
       'organization.state', 'attribution_text', 'preview_image', 'extras',
       'hvd_category', 'sample_rec

In [5]:
# Load the dataset from a parquet file into a DataFrame
df = pd.read_parquet(DATA_PATH)

# (Optional) Extract publisher information

# In the Zürich dataset, the publisher was extracted by splitting the 'identifier' field.
# However, Berlin’s CKAN metadata typically provides publisher information in a dedicated field,
# such as 'organization'. If your Berlin data includes this field, you can extract the publisher like so:
# if "organization" in df.columns:
#     # If 'organization' is a dictionary (as is common with CKAN), extract its title:
#     df["publisher"] = df["organization"].apply(
#         lambda org: org.get("title") if isinstance(org, dict) else org
#     )
# else:
#     print("No 'organization' field found; skipping publisher extraction.")

# (Optional) Filtering step: In the Zürich code, datasets were filtered to retain only those tagged as "ogd".

# In Berlin’s catalog (built on CKAN), all datasets are typically official Open Government Data.
# If you still need to filter (e.g., by a specific tag like "open-data"), adjust the lambda accordingly.
# if "tags" in df.columns:
#     # Example: filtering for datasets that include the term "open-data" in their tags
#     is_open_data = df.dropna(subset=["tags"]).tags.apply(
#         lambda x: "open-data" in x.lower()
#     )
#     df = df.loc[is_open_data[is_open_data].index]
# else:
#     print("No 'tags' column found; proceeding without tag-based filtering.")

# Reset the index of the DataFrame to ensure it is sequential after any filtering
# df.reset_index(drop=True, inplace=True)

# Check if the filtered DataFrame is empty; if so, raise an error; otherwise, print dataset count
# if len(df) == 0:
#     raise ValueError("No data retrieved.")
# else:
#     print(f"Retrieved {len(df)} datasets.")

In [6]:
def extract_active_names(tag_entry):
    tag_list = json.loads(tag_entry)
    return [tag['name'] for tag in tag_list if tag['state'] == 'active']

In [7]:
df['formatted_tags'] = df['tags'].apply(extract_active_names)

In [10]:
# Filter down dataset
df = df[(df['author'] != 'Stromnetz Berlin GmbH') & (~df['title'].str.contains('Sozialstatistisches Berichtswesen')) & (~df['title'].str.contains('Gesundheitsberichterstattung'))]

## Analyze and score metadata

#

# > **⚠️ Warning:** Processing a high number of datasets using an LLM might cost a bit of time and money. An alternative would be to use a batch API or use a local LLM. Please confer the docs of your LLM provider. Just a heads up!


In [11]:
# Be aware not to hit your OpenAI API rate limits with too many parallel requests.
n_parallel = 20

# Set a variable to specify the number of datasets to analyze. Default is None to process all datasets.
num_datasets_to_analyze = (
    None  # Change this value any number to None to process all datasets.
)

# Create a list of data rows to process in parallel, limited by the specified variable.
data_rows = [x[1] for x in list(df.iterrows())]

# If a number is specified, slice the data_rows; otherwise, use all datasets.
if num_datasets_to_analyze is not None:
    data_rows = data_rows[:num_datasets_to_analyze]

dataset_count = (
    num_datasets_to_analyze if num_datasets_to_analyze is not None else "all datasets"
)
print(
    f"Preparing to analyze {dataset_count} datasets in parallel with {n_parallel} workers."
)

# The analysis of the datasets will now begin using the specified number of parallel workers.
with ThreadPoolExecutor(max_workers=n_parallel) as executor:
    results = list(executor.map(do_full_analysis, data_rows))

# Print the LLM output for each dataset to check for empty XML responses
#for i, result in enumerate(results):
    #print(f"LLM output for dataset {i + 1}: {result}")

print("Analysis has been completed for all chosen datasets.")

Preparing to analyze all datasets datasets in parallel with 20 workers.
Analysis has been completed for all chosen datasets.


In [12]:
# parse instructor results
results_parsed = parse_instructor_results(results)
df_final = pd.concat([df.reset_index(), results_parsed], axis=1)
cols = [
    "id",
    "organization.title",
    "title",
    "notes",
    "formatted_tags",
    'geographical_coverage',
    'geographical_granularity',
    'author',
    'dateninhalt',
    'dateninhalt_score',
    'methodik',
    'methodik_score',
    'datenqualitaet',
    'datenqualitaet_score',
    'geographie',
    'geographie_score',
    'tag_qualitaet',
    'tag_qualitaet_score',
    'referenz',
    'referenz_score'
]
timestamp = datetime.now().strftime("%Y%m%d")
df_final[cols].to_excel(f"_results/metadata_analysis_{timestamp}.xlsx", index=False)

In [ ]:
# Create a DataFrame from the results of the analysis, specifying the column name "results_raw".
# This DataFrame will hold the raw results returned from the LLM analysis for each dataset.
results_parsed = pd.DataFrame(results, columns=["results_raw"])

# Apply the function 'parse_analysis_results' to each entry in the "results_raw" column.
# This function is expected to process the raw results and extract meaningful information.
# Be cautious as this step may involve additional LLM calls, which can incur costs.
results_parsed = results_parsed["results_raw"].apply(parse_analysis_results)

# Concatenate the list of DataFrames returned by 'parse_analysis_results' into a single DataFrame.
# This is necessary because 'parse_analysis_results' may return a DataFrame for each row,
# and we need to combine them into one cohesive DataFrame.
results_parsed = pd.concat(results_parsed.tolist(), axis=0)

# Reset the index of the concatenated DataFrame to ensure it is sequential after concatenation.
# This helps maintain a clean DataFrame structure for further processing.
results_parsed.reset_index(drop=True, inplace=True)

# Combine the original DataFrame 'df' with the parsed results DataFrame 'results_parsed'.
# This creates a final DataFrame that includes both the original dataset information and the analysis results.
df_final = pd.concat([df, results_parsed], axis=1)

# Define a list of columns that we want to retain in the final output.
# These columns include identifiers and various scores that were generated during the analysis.
cols = [
    "id",
    "organization.title",
    "title",
    "notes",
    "formatted_tags",
    'geographical_coverage',
    'geographical_granularity',
    'author',
    'dateninhalt',
    'dateninhalt_score',
    'methodik',
    'methodik_score',
    'datenqualitaet',
    'datenqualitaet_score',
    'geographie',
    'geographie_score',
    'tag_qualitaet',
    'tag_qualitaet_score',
    'referenz',
    'referenz_score'
]

# Generate a timestamp in the format YYYYMMDD to use in the filename for the output Excel file.
# This helps in organizing results by date.
timestamp = datetime.now().strftime("%Y%m%d")
df_final[cols].to_excel(f"_results/metadata_analysis_{timestamp}.xlsx", index=False)